data_bicicleta

In [1]:
import json
import os

# Funciones para evaluar la calidad de la descripción y el precio
def evaluar_descripcion(descripcion):
    score = 0
    # Evaluación por longitud de la descripción
    if len(descripcion) > 150:
        score += 5
    elif len(descripcion) > 100:
        score += 4
    elif len(descripcion) > 50:
        score += 3
    else:
        score += 1

    # Evaluación por palabras clave
    if "nuevo" in descripcion.lower() or "en perfecto estado" in descripcion.lower() or "sin abrir" in descripcion.lower():
        score += 2

    return score

def evaluar_precio(precio):
    if precio < 50:
        return 5
    elif precio < 150:
        return 4
    elif precio < 500:
        return 3
    elif precio < 1000:
        return 2
    else:
        return 1

# Leer productos desde un archivo JSON
def cargar_productos(ruta_archivo):
    try:
        with open(ruta_archivo, 'r', encoding='utf-8') as file:
            return json.load(file)
    except FileNotFoundError:
        print(f"Error: No se encontró el archivo en la ruta {ruta_archivo}")
        return []
    except json.JSONDecodeError:
        print("Error: El archivo no contiene un JSON válido")
        return []

# Guardar resultados en un archivo JSON
def guardar_resultados(resultados, ruta_salida):
    try:
        os.makedirs(os.path.dirname(ruta_salida), exist_ok=True)
        with open(ruta_salida, 'w', encoding='utf-8') as file:
            json.dump(resultados, file, indent=2, ensure_ascii=False)
        print(f"Archivo guardado exitosamente en: {ruta_salida}")
    except Exception as e:
        print(f"Error al guardar el archivo: {e}")

# Ruta del archivo JSON de entrada
ruta_archivo = '../data/coocked/json/data_bicicleta.json'  # Cambia esta línea a la ruta de tu archivo JSON

# Ruta del archivo JSON de salida
ruta_salida = '../data/coocked/json/scoring/data_bicicleta.json'  # Cambia esta línea a tu ruta de salida

# Cargar productos
productos = cargar_productos(ruta_archivo)

# Calcular puntaje para cada producto
resultados = []
if productos:
    for producto in productos:
        descripcion_score = evaluar_descripcion(producto["description"])
        precio_score = evaluar_precio(producto["amount"])
        puntaje_total = descripcion_score + precio_score
        
        # Asegurarse de que el puntaje no exceda 10
        resultados.append({
            "product_id": producto["id"],
            "user_id": producto["user_id"],
            "description": producto["description"],
            "price": f"{producto['amount']} {producto['currency']}",
            "taxonomy": producto["taxonomy"],
            "reservado": producto["reservado"],
            "envio": producto["envio"],
            "url_completa": producto["url_completa"],
            "ciudad": producto["ciudad"],
            "score": min(puntaje_total, 10)
        })

    # Guardar los resultados en el archivo JSON
    guardar_resultados(resultados, ruta_salida)

    # Imprimir resultados en la consola
    print(json.dumps(resultados, indent=2, ensure_ascii=False))
else:
    print("No se encontraron productos para procesar.")


Archivo guardado exitosamente en: ../data/coocked/json/scoring/data_bicicleta.json
[
  {
    "product_id": "pj9mp358kv6e",
    "user_id": "kmzn87ep2yzn",
    "description": "NUEVO SHIMANO DEORE SL-M6000 I-spec II 10 Velocidades. Pusador mando de cambio derecho. Trasero . Trigger .La maneta de cambio SL-M6000 I de SHIMANO DEORE ofrece un cambio y ligero y buena capacidad de respuesta para todos los estilos de ciclismo de montaña.\n\n",
    "price": "15 EUR",
    "taxonomy": "[{'id': 17000, 'name': 'Bikes', 'icon': 'bike'}, {'id': 10058, 'name': 'Bike components & spare parts', 'icon': None}, {'id': 10224, 'name': 'Components', 'icon': None}]",
    "reservado": false,
    "envio": true,
    "url_completa": "https://es.wallapop.com/item/nuevo-shimano-deore-sl-m6000-ispec-ii-10v-pulsador-1090284513",
    "ciudad": "Madrid",
    "score": 10
  },
  {
    "product_id": "xzo24vlyy469",
    "user_id": "wzynryvx8xj5",
    "description": "Giant Anthem 2<br> | Talla: M | Material: Aluminio | Freno

Cine, Libros y Música

In [2]:
import json
import random
import os

# Definición de precios de referencia para las categorías
precios_referencia = {
    "musical_instruments": [1200, 200, 450, 1100],
    "books": [75, 22, 50, 40, 20],
}

def evaluar_descripcion(descripcion):
    score = 0
    # Evaluar la longitud de la descripción con más variabilidad
    if len(descripcion) > 150:
        score += random.randint(5, 7)
    elif len(descripcion) > 100:
        score += random.randint(4, 6)
    elif len(descripcion) > 50:
        score += random.randint(3, 5)
    else:
        score += random.randint(1, 3)

    # Evaluar palabras clave con puntuación variable
    palabras_clave = {
        "original": random.randint(1, 3),
        "nuevo": random.randint(1, 3),
        "estado": random.randint(1, 3),
        "marca": random.randint(1, 2),
        "modelo": random.randint(1, 2)
    }
    
    score += sum(palabras_clave[palabra] for palabra in palabras_clave if palabra in descripcion.lower())

    return score

def evaluar_precio(precio, categoria):
    precio_medio = sum(precios_referencia[categoria]) / len(precios_referencia[categoria])
    
    # Introducir más variabilidad en la evaluación de precio
    diferencia_porcentual = abs(precio - precio_medio) / precio_medio * 100
    
    if diferencia_porcentual < 10:
        return random.randint(1, 2)  # Precio muy cercano
    elif precio < precio_medio:
        return random.randint(2, 3)  # Precio bajo
    else:
        return random.randint(0, 1)  # Precio alto

def calcular_puntaje(producto):
    # Más flexibilidad en la categoría
    if "guitarra" in producto["description"].lower():
        categoria = "musical_instruments"
    elif "libro" in producto["description"].lower():
        categoria = "books"
    else:
        categoria = random.choice(list(precios_referencia.keys()))
    
    descripcion_score = evaluar_descripcion(producto["description"])
    precio_score = evaluar_precio(producto["amount"], categoria)

    puntaje_total = descripcion_score + precio_score
    return min(puntaje_total, 10)  # Asegurarse de que el puntaje no supere 10

def cargar_productos(ruta_archivo):
    try:
        with open(ruta_archivo, 'r', encoding='utf-8') as file:
            return json.load(file)
    except FileNotFoundError:
        print(f"Error: No se encontró el archivo en la ruta {ruta_archivo}")
        return []
    except json.JSONDecodeError:
        print("Error: El archivo no contiene un JSON válido")
        return []

def guardar_resultados(resultados, ruta_salida):
    try:
        os.makedirs(os.path.dirname(ruta_salida), exist_ok=True)
        with open(ruta_salida, 'w', encoding='utf-8') as file:
            json.dump(resultados, file, indent=2, ensure_ascii=False)
        print(f"Archivo guardado exitosamente en: {ruta_salida}")
    except Exception as e:
        print(f"Error al guardar el archivo: {e}")

# Ruta del archivo JSON de entrada
ruta_archivo = r'..\data\coocked\json\data_cine_libros_musica.json'

# Ruta del archivo JSON de salida
ruta_salida = r'..\data\coocked\json\scoring\data_cine_libros_musica.json'

# Cargar productos
productos = cargar_productos(ruta_archivo)

# Calcular puntajes para cada producto
resultados = []
if productos:
    for producto in productos:
        puntaje = calcular_puntaje(producto)
        resultados.append({
            "product_id": producto["id"],
            "user_id": producto["user_id"],
            "description": producto["description"],
            "price": f"{producto['amount']} {producto['currency']}",
            "ciudad": producto["ciudad"],
            "taxonomy": producto["taxonomy"],
            "reservado": producto["reservado"],
            "envio": producto["envio"],
            "url_completa": producto["url_completa"],
            "score": puntaje
        })

    # Guardar los resultados en el archivo JSON
    guardar_resultados(resultados, ruta_salida)

    # Imprimir resultados en la consola
    print(json.dumps(resultados, indent=2, ensure_ascii=False))
else:
    print("No se encontraron productos para procesar.")


Archivo guardado exitosamente en: ..\data\coocked\json\scoring\data_cine_libros_musica.json
[
  {
    "product_id": "0j24d30qqezy",
    "user_id": "qjwy7dk1wqzo",
    "description": "Está en excelente estado, tiene algunas pequeñas marcas en el cuerpo. Mira las fotos para más detalles.\n¡Solo recogida en persona! Estoy en el centro de Madrid!\nCuerpo: Fresno\nColor: Blanco translúcido\nAcabado del cuerpo: Uretano brillante\nMástil: Arce\nAcabado del mástil: Uretano satinado en la parte trasera\nPerfil del mástil: Forma “C” moderna\nDiapasón: Palo de rosa\nRadio: 9.5\nNúmero de trastes: 22\nTipo de trastes: Medium Jumbo\nPastillas del mástil: Noiseless Vintage Strat\nHardware: Dorado\nClavijeros: Deluxe cerrados con bloqueo\nPuente: Trémolo sincronizado de dos puntos, con selletas vintage\nIncluye stuche: Estuche moldeado Fender",
    "price": "1200 EUR",
    "ciudad": "Madrid",
    "taxonomy": "[{'id': 12463, 'name': 'Movies, books & music', 'icon': 'bookshelf'}, {'id': 24001, 'name': 

Coleccionismo

In [3]:
import json
import random
import os

# Definición de precios de referencia
precios_referencia = {
    "collectibles": [0.22, 5.0, 20.0, 4.0, 12.0, 10.0, 30.0, 40.0, 225.0, 4400.0],
}

def evaluar_descripcion(descripcion):
    score = 0
    # Variar la puntuación de longitud
    if len(descripcion) > 150:
        score += random.randint(4, 6)
    elif len(descripcion) > 100:
        score += random.randint(3, 5)
    elif len(descripcion) > 50:
        score += random.randint(2, 4)
    else:
        score += random.randint(1, 3)

    # Puntuación variable para palabras clave
    palabras_clave = {
        "nuevo": random.randint(1, 2),
        "original": random.randint(1, 2),
        "estado": random.randint(1, 2),
        "colección": random.randint(1, 2),
        "coleccionista": random.randint(1, 2)
    }
    
    score += sum(palabras_clave[palabra] for palabra in palabras_clave if palabra in descripcion.lower())

    return score

def evaluar_precio(precio, categoria):
    precio_medio = sum(precios_referencia[categoria]) / len(precios_referencia[categoria])
    
    # Evaluación más detallada del precio
    diferencia_porcentual = abs(precio - precio_medio) / precio_medio * 100
    
    if diferencia_porcentual < 10:
        return random.randint(1, 2)  # Precio muy cercano
    elif precio < precio_medio:
        return random.randint(2, 3)  # Precio bajo
    else:
        return random.randint(0, 1)  # Precio alto

def calcular_puntaje(producto):
    categoria = "collectibles"
    descripcion_score = evaluar_descripcion(producto["description"])
    precio_score = evaluar_precio(producto["amount"], categoria)

    puntaje_total = descripcion_score + precio_score
    return min(puntaje_total, 10)

def cargar_productos(ruta_archivo):
    try:
        with open(ruta_archivo, 'r', encoding='utf-8') as file:
            return json.load(file)
    except FileNotFoundError:
        print(f"Error: No se encontró el archivo en la ruta {ruta_archivo}")
        return []
    except json.JSONDecodeError:
        print("Error: El archivo no contiene un JSON válido")
        return []

def guardar_resultados(resultados, ruta_salida):
    try:
        os.makedirs(os.path.dirname(ruta_salida), exist_ok=True)
        with open(ruta_salida, 'w', encoding='utf-8') as file:
            json.dump(resultados, file, indent=2, ensure_ascii=False)
        print(f"Archivo guardado exitosamente en: {ruta_salida}")
    except Exception as e:
        print(f"Error al guardar el archivo: {e}")

# Ruta del archivo JSON de entrada
ruta_archivo = r'../data/coocked/json/data_coleccionismo.json'

# Ruta del archivo JSON de salida
ruta_salida = r'../data/coocked/json/scoring/data_coleccionismo.json'

# Cargar productos
productos = cargar_productos(ruta_archivo)

# Calcular puntajes para cada producto
resultados = []
if productos:
    for producto in productos:
        puntaje = calcular_puntaje(producto)
        resultados.append({
            "product_id": producto["id"],
            "user_id": producto["user_id"],
            "description": producto["description"],
            "price": f"{producto['amount']} {producto['currency']}",
            "ciudad": producto["ciudad"],
            "taxonomy": producto["taxonomy"],
            "reservado": producto["reservado"],
            "envio": producto["envio"],
            "url_completa": producto["url_completa"],
            "score": puntaje
        })

    # Guardar los resultados en el archivo JSON
    guardar_resultados(resultados, ruta_salida)

    # Imprimir resultados en la consola
    print(json.dumps(resultados, indent=2, ensure_ascii=False))
else:
    print("No se encontraron productos para procesar.")


Archivo guardado exitosamente en: ../data/coocked/json/scoring/data_coleccionismo.json
[
  {
    "product_id": "w6745o94476x",
    "user_id": "qnzx705vm7j2",
    "description": "Se envían protegidos con fundas de plástico y pequeños sobres, en caso de que sea balón de oro momento míticos, extra se envía también con un toploader, solo bizum, pedido minimo 1 euro aunque solo sea 1 codigo.\nPrecios:\nNormales: 0,25\nescudos: 0,50\nVamos: 0,50\nGuantes de oro: 0,40\nDiamantes: 0,35\nAir force: 0,45\nInfluencers: 0,45\nImpactos: 0,45\nSuper cracks: 1 euro\nnew master 11: 1,5 euros\nCampeon card: 6\nBalón de oro: 10 euros\nInvencible: 10 euros\nMomentum: 20 euros\nMíticos: 40 euros\nLamine yamal extra gold: 45 euros\nLos precios pueden variar según el jugador (lamine yamal o mbappe) sobre todo en balones de oro y momentum, míticos",
    "price": "0.22 EUR",
    "ciudad": "Madrid",
    "taxonomy": "[{'id': 18000, 'name': 'Collectibles & Art', 'icon': 'collecting'}, {'id': 10076, 'name': 'Spor

Construcción

In [4]:
import json
import random
import os

# Definición de precios de referencia
precios_referencia = {
    "construction": [0, 30, 5, 1, 50, 22, 175, 600, 900, 372],
}

def evaluar_descripcion(descripcion):
    score = 0
    # Variar la puntuación de longitud
    if len(descripcion) > 150:
        score += random.randint(4, 6)
    elif len(descripcion) > 100:
        score += random.randint(3, 5)
    elif len(descripcion) > 50:
        score += random.randint(2, 4)
    else:
        score += random.randint(1, 3)

    # Puntuación variable para palabras clave
    palabras_clave = {
        "nuevo": random.randint(1, 2),
        "original": random.randint(1, 2),
        "experiencia": random.randint(1, 2),
        "garantía": random.randint(1, 2),
        "calidad": random.randint(1, 2),
        "profesional": random.randint(1, 2),
        "presupuesto": random.randint(1, 2)
    }
    
    score += sum(palabras_clave[palabra] for palabra in palabras_clave if palabra in descripcion.lower())

    return score

def evaluar_precio(precio, categoria):
    precio_medio = sum(precios_referencia[categoria]) / len(precios_referencia[categoria])
    
    # Evaluación más detallada del precio
    diferencia_porcentual = abs(precio - precio_medio) / precio_medio * 100
    
    if diferencia_porcentual < 10:
        return random.randint(1, 2)  # Precio muy cercano
    elif precio < precio_medio:
        return random.randint(2, 3)  # Precio bajo
    else:
        return random.randint(0, 1)  # Precio alto

def calcular_puntaje(producto):
    categoria = "construction"
    descripcion_score = evaluar_descripcion(producto["description"])
    precio_score = evaluar_precio(producto["amount"], categoria)

    puntaje_total = descripcion_score + precio_score
    return min(puntaje_total, 10)

def cargar_productos(ruta_archivo):
    try:
        with open(ruta_archivo, 'r', encoding='utf-8') as file:
            return json.load(file)
    except FileNotFoundError:
        print(f"Error: No se encontró el archivo en la ruta {ruta_archivo}")
        return []
    except json.JSONDecodeError:
        print("Error: El archivo no contiene un JSON válido")
        return []

def guardar_resultados(resultados, ruta_salida):
    try:
        os.makedirs(os.path.dirname(ruta_salida), exist_ok=True)
        with open(ruta_salida, 'w', encoding='utf-8') as file:
            json.dump(resultados, file, indent=2, ensure_ascii=False)
        print(f"Archivo guardado exitosamente en: {ruta_salida}")
    except Exception as e:
        print(f"Error al guardar el archivo: {e}")

# Ruta del archivo JSON de entrada
ruta_archivo = r'../data/coocked/json/data_construccion_reforma.json'

# Ruta del archivo JSON de salida
ruta_salida = r'../data/coocked/json/scoring/data_construccion_reforma.json'

# Cargar productos
productos = cargar_productos(ruta_archivo)

# Calcular puntajes para cada producto
resultados = []
if productos:
    for producto in productos:
        puntaje = calcular_puntaje(producto)
        resultados.append({
            "product_id": producto["id"],
            "user_id": producto["user_id"],
            "description": producto["description"],
            "price": f"{producto['amount']} {producto['currency']}",
            "ciudad": producto["ciudad"],
            "taxonomy": producto["taxonomy"],
            "reservado": producto["reservado"],
            "envio": producto["envio"],
            "url_completa": producto["url_completa"],
            "score": puntaje
        })

    # Guardar los resultados en el archivo JSON
    guardar_resultados(resultados, ruta_salida)

    # Imprimir resultados en la consola
    print(json.dumps(resultados, indent=2, ensure_ascii=False))
else:
    print("No se encontraron productos para procesar.")


Archivo guardado exitosamente en: ../data/coocked/json/scoring/data_construccion_reforma.json
[
  {
    "product_id": "nz04385nmrjo",
    "user_id": "kmzn3g78wyjn",
    "description": "Grupo Genesis, nos encargamos de hacer todo tipo de reformas, ya sean integrales o cambiar solo un baño, contamos con mas de 20 años de experiencia en el sector, y un gran equipo de profesionales, le atenderemos desde el comienzo de la obra hasta el final, no dude en ponerse en contacto con nosotros, atendemos por telefono, WhatsApp o Email, puede venir a visitarnos a nuestras oficinas o podemos desplazarnos al lugar deseado.",
    "price": "0 EUR",
    "ciudad": "Madrid",
    "taxonomy": "[{'id': 19000, 'name': 'Construction', 'icon': 'building'}, {'id': 10146, 'name': 'Others', 'icon': None}]",
    "reservado": false,
    "envio": false,
    "url_completa": "https://es.wallapop.com/item/todo-tipo-de-reformas-1077247927",
    "score": 10
  },
  {
    "product_id": "nzx4xl21kej2",
    "user_id": "36ewdy0

Deporte

In [5]:
import json
import random
import os

# Definición de precios de referencia
precios_referencia = {
    "sports_leisure": [215.0, 98.0, 225.0, 20.0, 269.0, 15.0, 6.0, 300.0, 245.0],
}

def evaluar_descripcion(descripcion):
    score = 0
    # Variar la puntuación de longitud
    if len(descripcion) > 150:
        score += random.randint(4, 6)
    elif len(descripcion) > 100:
        score += random.randint(3, 5)
    elif len(descripcion) > 50:
        score += random.randint(2, 4)
    else:
        score += random.randint(1, 3)

    # Puntuación variable para palabras clave
    palabras_clave = {
        "nuevo": random.randint(1, 2),
        "original": random.randint(1, 2),
        "experiencia": random.randint(1, 2),
        "garantía": random.randint(1, 2),
        "calidad": random.randint(1, 2),
        "profesional": random.randint(1, 2),
        "presupuesto": random.randint(1, 2)
    }
    
    score += sum(palabras_clave[palabra] for palabra in palabras_clave if palabra in descripcion.lower())

    return score

def evaluar_precio(precio, categoria):
    precio_medio = sum(precios_referencia[categoria]) / len(precios_referencia[categoria])
    
    # Evaluación más detallada del precio
    diferencia_porcentual = abs(precio - precio_medio) / precio_medio * 100
    
    if diferencia_porcentual < 10:
        return random.randint(1, 2)  # Precio muy cercano
    elif precio < precio_medio:
        return random.randint(2, 3)  # Precio bajo
    else:
        return random.randint(0, 1)  # Precio alto

def calcular_puntaje(producto):
    categoria = "sports_leisure"
    descripcion_score = evaluar_descripcion(producto["description"])
    precio_score = evaluar_precio(producto["amount"], categoria)

    puntaje_total = descripcion_score + precio_score
    return min(puntaje_total, 10)

def cargar_productos(ruta_archivo):
    try:
        with open(ruta_archivo, 'r', encoding='utf-8') as file:
            return json.load(file)
    except FileNotFoundError:
        print(f"Error: No se encontró el archivo en la ruta {ruta_archivo}")
        return []
    except json.JSONDecodeError:
        print("Error: El archivo no contiene un JSON válido")
        return []

def guardar_resultados(resultados, ruta_salida):
    try:
        os.makedirs(os.path.dirname(ruta_salida), exist_ok=True)
        with open(ruta_salida, 'w', encoding='utf-8') as file:
            json.dump(resultados, file, indent=2, ensure_ascii=False)
        print(f"Archivo guardado exitosamente en: {ruta_salida}")
    except Exception as e:
        print(f"Error al guardar el archivo: {e}")

# Ruta del archivo JSON de entrada
ruta_archivo = r'../data/coocked/json/data_deporte_ocio.json'

# Ruta del archivo JSON de salida
ruta_salida = r'../data/coocked/json/scoring/data_deporte_ocio.json'

# Cargar productos
productos = cargar_productos(ruta_archivo)

# Calcular puntajes para cada producto
resultados = []
if productos:
    for producto in productos:
        puntaje = calcular_puntaje(producto)
        resultados.append({
            "product_id": producto["id"],
            "user_id": producto["user_id"],
            "description": producto["description"],
            "price": f"{producto['amount']} {producto['currency']}",
            "ciudad": producto["ciudad"],
            "taxonomy": producto["taxonomy"],
            "reservado": producto["reservado"],
            "envio": producto["envio"],
            "url_completa": producto["url_completa"],
            "score": puntaje
        })

    # Guardar los resultados en el archivo JSON
    guardar_resultados(resultados, ruta_salida)

    # Imprimir resultados en la consola
    print(json.dumps(resultados, indent=2, ensure_ascii=False))
else:
    print("No se encontraron productos para procesar.")


Archivo guardado exitosamente en: ../data/coocked/json/scoring/data_deporte_ocio.json
[
  {
    "product_id": "vjrq82dxgwzk",
    "user_id": "8j3m37p25l69",
    "description": "chaleco de buceo ideal tanto para iniciarte como si ya tienes algo de experiencia en el buceo recreativo.\nSon chalecos nuevos en su caja.\nNos queda uno talla S y una talla M",
    "price": "215.0 EUR",
    "ciudad": "Madrid",
    "taxonomy": "[{'id': 12579, 'name': 'Sports & leisure', 'icon': 'ball'}, {'id': 10104, 'name': 'Others', 'icon': None}]",
    "reservado": false,
    "envio": true,
    "url_completa": "https://es.wallapop.com/item/chaleco-cressi-start-1080297091",
    "score": 9
  },
  {
    "product_id": "qzmed34lvojv",
    "user_id": "8ejk85n51yjx",
    "description": "Ordenador GPS \n- Competiciones en recorrido mediante segmentos Garmin Connect™\n- Navegación y puntos de interés en carretera y fuera de carretera \n- Hasta 3 opciones de rutas de ida y vuelta\n- Cargas instantáneas a Garmin Connect

Hogar y Jardin

In [16]:
import json
import os

# Definición de precios de referencia
precios_referencia = {
    "home_garden": [25.0, 100.0, 45.0, 140.0, 70.0, 125.0, 50.0, 15.0, 150.0, 10.0, 13.0, 90.0, 220.0, 50.0, 120.0, 150.0, 4.0, 600.0, 45.0, 35.0, 190.0, 4.99, 100.0],
    # ... (otros precios de referencia)
}

# Función para evaluar la descripción
def evaluar_descripcion(descripcion):
    score = 0
    if len(descripcion) > 150:
        score += 5
    elif len(descripcion) > 100:
        score += 4
    elif len(descripcion) > 50:
        score += 3
    else:
        score += 1

    if any(keyword in descripcion.lower() for keyword in ["nuevo", "artesanal", "perfecto", "calidad"]):
        score += 2
    if any(keyword in descripcion.lower() for keyword in ["garantía", "personalizado", "decora"]):
        score += 1

    return score

# Función para evaluar el precio
def evaluar_precio(precio, categoria):
    precio_medio = sum(precios_referencia[categoria]) / len(precios_referencia[categoria])
    if precio < precio_medio:
        return 3  # Bajo precio
    elif precio == precio_medio:
        return 1  # Precio promedio
    else:
        return 0  # Precio alto

# Función para calcular el puntaje total
def calcular_puntaje(producto):
    categoria = "home_garden"  # Asumimos que todos son del hogar y jardín
    descripcion_score = evaluar_descripcion(producto["description"])
    precio_score = evaluar_precio(producto["amount"], categoria)

    puntaje_total = descripcion_score + precio_score
    return min(puntaje_total, 10)

# Función para guardar resultados en un archivo JSON
def guardar_resultados(resultados, ruta_salida):
    try:
        os.makedirs(os.path.dirname(ruta_salida), exist_ok=True)
        with open(ruta_salida, 'w', encoding='utf-8') as file:
            json.dump(resultados, file, indent=2, ensure_ascii=False)
        print(f"Archivo guardado exitosamente en: {ruta_salida}")
    except Exception as e:
        print(f"Error al guardar el archivo: {e}")

# Ruta del archivo JSON de entrada
ruta_archivo = '..\\data\\coocked\\json\\data_hogar_jardin.json'  # Cambia esta línea a la ruta de tu archivo JSON

# Ruta del archivo JSON de salida
ruta_salida = '..\\data\\coocked\\json\\scoring\\data_hogar_jardin.json'  # Cambia esta línea a tu ruta de salida

# Leer productos desde un archivo JSON
def cargar_productos(ruta_archivo):
    try:
        with open(ruta_archivo, 'r', encoding='utf-8') as file:
            return json.load(file)
    except FileNotFoundError:
        print(f"Error: No se encontró el archivo en la ruta {ruta_archivo}")
        return []
    except json.JSONDecodeError:
        print("Error: El archivo no contiene un JSON válido")
        return []

# Procesar productos
productos = cargar_productos(ruta_archivo)

resultados = []
if productos:
    for producto in productos:
        puntaje = calcular_puntaje(producto)
        resultados.append({
            "product_id": producto["id"],
            "user_id": producto["user_id"],
            "description": producto["description"],
            "price": f"{producto['amount']} {producto['currency']}",
            "ciudad": producto["ciudad"],
            "taxonomy": producto["taxonomy"],
            "reservado": producto["reservado"],
            "envio": producto["envio"],
            "url_completa": producto["url_completa"],
            "score": puntaje
        })

    # Guardar los resultados en un archivo JSON
    guardar_resultados(resultados, ruta_salida)
else:
    print("No se encontraron productos para procesar.")


Archivo guardado exitosamente en: ..\data\coocked\json\scoring\data_hogar_jardin.json


Electrodoesticos

In [7]:
import json

# Funciones para evaluar la calidad de la descripción y el precio
def evaluar_descripcion(descripcion):
    score = 0
    # Evaluación por longitud de la descripción
    if len(descripcion) > 150:
        score += 5
    elif len(descripcion) > 100:
        score += 4
    elif len(descripcion) > 50:
        score += 3
    else:
        score += 1

    # Evaluación por palabras clave
    if "nuevo" in descripcion.lower() or "en perfecto estado" in descripcion.lower() or "sin estrenar" in descripcion.lower():
        score += 2

    return score

def evaluar_precio(precio):
    if precio < 50:
        return 5
    elif precio < 150:
        return 4
    elif precio < 500:
        return 3
    elif precio < 1000:
        return 2
    else:
        return 1

# Leer productos desde un archivo JSON
def cargar_productos(ruta_archivo):
    with open(ruta_archivo, 'r', encoding='utf-8') as file:
        return json.load(file)

# Guardar productos procesados en un archivo JSON
def guardar_resultados(resultados, ruta_salida):
    try:
        with open(ruta_salida, 'w', encoding='utf-8') as file:
            json.dump(resultados, file, indent=2, ensure_ascii=False)
        print(f"Archivo guardado exitosamente en: {ruta_salida}")
    except Exception as e:
        print(f"Error al guardar el archivo: {e}")

# Ruta del archivo JSON de entrada
ruta_archivo = '..\\data\\coocked\\json\\data_electrodomesticos.json'  # Cambia esta línea a la ruta de tu archivo JSON

# Ruta del archivo JSON de salida
ruta_salida = '..\\data\\coocked\\json\\scoring\\data_electrodomesticos.json'  # Cambia esta línea a tu ruta de salida

# Cargar productos
productos = cargar_productos(ruta_archivo)

# Calcular puntaje para cada producto
resultados = []
for producto in productos:
    descripcion_score = evaluar_descripcion(producto["description"])
    precio_score = evaluar_precio(producto["amount"])
    puntaje_total = min(descripcion_score + precio_score, 10)  # Limitar el puntaje a un máximo de 10
    
    resultados.append({
        "product_id": producto["id"],
        "user_id": producto["user_id"],
        "description": producto["description"],
        "price": f"{producto['amount']} {producto['currency']}",
        "ciudad": producto["ciudad"],
        "taxonomy": producto["taxonomy"],
        "reservado": producto["reservado"],
        "envio": producto["envio"],
        "url_completa": producto["url_completa"],
        "score": puntaje_total
    })

# Guardar resultados en el archivo especificado
guardar_resultados(resultados, ruta_salida)


Archivo guardado exitosamente en: ..\data\coocked\json\scoring\data_electrodomesticos.json


Industria_agricultura

In [8]:
import json
import statistics

# Definición de precios de referencia
precios_referencia = {
    "agriculture_industrial": [5500, 1, 395, 13000, 750, 690, 30, 480, 2400, 250, 18, 1800, 0, 50, 280, 145, 145, 145, 145, 340],
    # Aquí se pueden añadir más categorías según sea necesario
}

def evaluar_descripcion(descripcion):
    score = 0
    # Evaluación por longitud de la descripción
    if len(descripcion) > 150:
        score += 5
    elif len(descripcion) > 100:
        score += 4
    elif len(descripcion) > 50:
        score += 3
    else:
        score += 1

    # Evaluación por palabras clave principales
    palabras_clave_principales = ["nuevo", "artesanal", "calidad", "garantía"]
    if any(keyword in descripcion.lower() for keyword in palabras_clave_principales):
        score += 2

    # Evaluación por palabras clave secundarias
    palabras_clave_secundarias = ["personalizado", "detalles", "características"]
    if any(keyword in descripcion.lower() for keyword in palabras_clave_secundarias):
        score += 1

    return score

def evaluar_precio(precio, categoria):
    try:
        precio_medio = statistics.mean(precios_referencia[categoria])
        if precio < precio_medio:
            return 3  # Precio competitivo
        elif precio == precio_medio:
            return 1  # Precio promedio
        else:
            return 0  # Precio por encima del promedio
    except KeyError:
        return 1  # Valor por defecto si la categoría no existe

def cargar_productos(ruta_archivo):
    try:
        with open(ruta_archivo, 'r', encoding='utf-8') as file:
            return json.load(file)
    except FileNotFoundError:
        print(f"Error: No se encontró el archivo en la ruta {ruta_archivo}")
        return []
    except json.JSONDecodeError:
        print("Error: El archivo no contiene un JSON válido")
        return []

def procesar_productos(productos):
    resultados = []
    for producto in productos:
        # Asumimos categoría agriculture_industrial como predeterminada
        categoria = "agriculture_industrial"
        
        descripcion_score = evaluar_descripcion(producto["description"])
        precio_score = evaluar_precio(producto["amount"], categoria)
        puntaje_total = min(descripcion_score + precio_score, 10)
        
        resultados.append({
            "product_id": producto["id"],
            "description": producto["description"],
            "price": f"${producto['amount']}",
            "score": puntaje_total
        })
    return resultados

import json
import statistics
import os

# [Resto del código se mantiene igual hasta la función main()]

def main():
    # Ruta del archivo JSON de entrada
    ruta_archivo = '..\data\coocked\json\data_industria_agricultura.json'
    
    # Obtener el nombre base del archivo sin extensión y sin ruta
    nombre_base = os.path.basename(ruta_archivo)
    nombre_sin_extension = os.path.splitext(nombre_base)[0]
    
    # Crear el nombre del archivo de salida
    nombre_salida = f"{nombre_sin_extension}.json"
    
    # Crear la ruta completa de salida
    ruta_salida = os.path.join('..', 'data', 'coocked', 'json', 'scoring', nombre_salida)
    
    # Asegurar que el directorio de salida existe
    os.makedirs(os.path.dirname(ruta_salida), exist_ok=True)
    
    # Cargar productos
    productos = cargar_productos(ruta_archivo)
    
    if productos:
        # Procesar productos y obtener resultados
        resultados = procesar_productos(productos)
        
        # Generar salida en formato JSON
        output_json = json.dumps(resultados, indent=2, ensure_ascii=False)
        print(output_json)
        
        # Guardar resultados en el archivo
        with open(ruta_salida, 'w', encoding='utf-8') as f:
            json.dump(resultados, f, indent=2, ensure_ascii=False)
        print(f"Archivo guardado exitosamente en: {ruta_salida}")

if __name__ == "__main__":
    main()

[
  {
    "product_id": "e658kql8yk6o",
    "description": "Se vende fundidora como nueva con vacio y argon  ( 50 fundiciones) mas mezcladora revestimiento ",
    "price": "$5500",
    "score": 3
  },
  {
    "product_id": "xzo24vrl0269",
    "description": "excelentes condiciones",
    "price": "$1",
    "score": 4
  },
  {
    "product_id": "36e17wlx48jd",
    "description": "Podadoras Ht 130 km 94 y Ht 100\nComo nuevas \n",
    "price": "$395",
    "score": 4
  },
  {
    "product_id": "0j2425y2omzy",
    "description": "Sembradora airsem gil de 6 metros de ancho con preparador e indicadores en perfecto estado.",
    "price": "$13000",
    "score": 3
  },
  {
    "product_id": "3zl8wgnyk46x",
    "description": "vendo taladro de Columna en buen estado y funcionando con boton de emergencia en normativa. ",
    "price": "$750",
    "score": 6
  },
  {
    "product_id": "8j34dqvvg269",
    "description": "Se vende:\nPrensa aceite plato 50cm,\nUna plancha hueca,\nUna docena de capachos 

Inmobilaria.

In [9]:
import json
import statistics
import os

# Definición de precios de referencia
precios_referencia = {
    "real_estate": [17950, 990, 95, 12900, 50, 1600, 190, 15000, 2200, 1700, 2500, 3000],
    # Aquí se pueden añadir más categorías según sea necesario
}

def evaluar_descripcion(descripcion):
    score = 0
    # Evaluación por longitud de la descripción
    if len(descripcion) > 150:
        score += 5
    elif len(descripcion) > 100:
        score += 4
    elif len(descripcion) > 50:
        score += 3
    else:
        score += 1

    # Evaluación por palabras clave principales
    palabras_clave_principales = ["nuevo", "artesanal", "calidad", "garantía"]
    if any(keyword in descripcion.lower() for keyword in palabras_clave_principales):
        score += 2

    # Evaluación por palabras clave secundarias
    palabras_clave_secundarias = ["personalizado", "detalles", "características"]
    if any(keyword in descripcion.lower() for keyword in palabras_clave_secundarias):
        score += 1

    return score

def evaluar_precio(precio, categoria):
    try:
        precio_medio = statistics.mean(precios_referencia[categoria])
        if precio < precio_medio:
            return 3  # Precio competitivo
        elif precio == precio_medio:
            return 1  # Precio promedio
        else:
            return 0  # Precio por encima del promedio
    except KeyError:
        return 1  # Valor por defecto si la categoría no existe

def cargar_productos(ruta_archivo):
    try:
        with open(ruta_archivo, 'r', encoding='utf-8') as file:
            return json.load(file)
    except FileNotFoundError:
        print(f"Error: No se encontró el archivo en la ruta {ruta_archivo}")
        return []
    except json.JSONDecodeError:
        print("Error: El archivo no contiene un JSON válido")
        return []

def procesar_productos(productos):
    resultados = []
    for producto in productos:
        # Asumimos categoría real_estate como predeterminada
        categoria = "real_estate"
        
        descripcion_score = evaluar_descripcion(producto["description"])
        precio_score = evaluar_precio(producto["amount"], categoria)
        puntaje_total = min(descripcion_score + precio_score, 10)
        
        resultados.append({
            "product_id": producto["id"],
            "description": producto["description"],
            "price": f"${producto['amount']}",
            "score": puntaje_total
        })
    return resultados

def main():
    # Ruta del archivo JSON de entrada
    ruta_archivo = '..\data\coocked\json\data_inmobilaria.json'
    
    # Obtener el nombre base del archivo sin extensión y sin ruta
    nombre_base = os.path.basename(ruta_archivo)
    nombre_sin_extension = os.path.splitext(nombre_base)[0]
    
    # Crear el nombre del archivo de salida
    nombre_salida = f"{nombre_sin_extension}.json"
    
    # Crear la ruta completa de salida
    ruta_salida = os.path.join('..', 'data', 'coocked', 'json', 'scoring', nombre_salida)
    
    # Asegurar que el directorio de salida existe
    os.makedirs(os.path.dirname(ruta_salida), exist_ok=True)
    
    # Cargar productos
    productos = cargar_productos(ruta_archivo)
    
    if productos:
        # Procesar productos y obtener resultados
        resultados = procesar_productos(productos)
        
        # Generar salida en formato JSON
        output_json = json.dumps(resultados, indent=2, ensure_ascii=False)
        print(output_json)
        
        # Guardar resultados en el archivo
        with open(ruta_salida, 'w', encoding='utf-8') as f:
            json.dump(resultados, f, indent=2, ensure_ascii=False)
        print(f"Archivo guardado exitosamente en: {ruta_salida}")

if __name__ == "__main__":
    main()

[
  {
    "product_id": "3zl8wp8ewp6x",
    "description": "Se vende excelente mobil home casa móvil seminueva poco uso de la marca de mobilhome y casas prefabricadas Ohara distribuida en 3 dormitorios, cocina, salón comedor, WC y baño. Climalit en ventanas. Venga a verla a Aranjuez (Madrid) Tenemos más casas moviles en stock en Madrid, Alicante y Euskadi. Mide 8x4 m. Financiamos la compra sin entrada hasta en 120 meses. Abrimos todos los días del año, festivos incluidos. ",
    "price": "$17950",
    "score": 5
  },
  {
    "product_id": "0j247rr1nvzy",
    "description": "¡Descubre tu nuevo hogar en el corazón de Madrid! Este magnífico estudio, ubicado en la vibrante zona de Huertas-Cortes, te ofrece un espacio amplio y luminoso donde disfrutar de la vida urbana. A tan solo un paso del Paseo de la Castellana y la Plaza de Santa Ana, tendrás todo lo que necesitas al alcance de la mano. El edificio cuenta con ascensor, facilitando tu día a día, y el estudio está listo para entrar a viv

Moda Acesorios.

In [10]:
import json
import os

def evaluar_descripcion(descripcion):
   score = 0
   # Evaluación por longitud de la descripción
   if len(descripcion) > 150:
       score += 5
   elif len(descripcion) > 100:
       score += 4
   elif len(descripcion) > 50:
       score += 3
   else:
       score += 1

   # Evaluación por palabras clave principales
   if "nuevo" in descripcion.lower():
       score += 2
   
   # Evaluación por palabras clave secundarias 
   if "original" in descripcion.lower() or "autentica" in descripcion.lower():
       score += 1

   return score

def evaluar_precio(precio):
   if precio < 50:
       return 5
   elif precio < 150:
       return 4
   elif precio < 300:
       return 3
   elif precio < 500:
       return 2
   else:
       return 1

def cargar_productos(ruta_archivo):
   try:
       with open(ruta_archivo, 'r', encoding='utf-8') as file:
           return json.load(file)
   except FileNotFoundError:
       print(f"Error: No se encontró el archivo en la ruta {ruta_archivo}")
       return []
   except json.JSONDecodeError:
       print("Error: El archivo no contiene un JSON válido")
       return []

def procesar_productos(productos):
   resultados = []
   for producto in productos:
       descripcion_score = evaluar_descripcion(producto["description"])
       precio_score = evaluar_precio(producto["amount"])
       puntaje_total = min(descripcion_score + precio_score, 10)
       
       resultados.append({
           "product_id": producto["id"],
           "description": producto["description"],
           "price": f"${producto['amount']}",
           "score": puntaje_total
       })
   return resultados

def main():
   # Ruta del archivo JSON de entrada
   ruta_archivo = '..\data\coocked\json\data_moda_accesorios.json'
   
   # Obtener el nombre base del archivo sin extensión y sin ruta
   nombre_base = os.path.basename(ruta_archivo)
   nombre_sin_extension = os.path.splitext(nombre_base)[0]
   
   # Crear el nombre del archivo de salida
   nombre_salida = f"{nombre_sin_extension}.json"
   
   # Crear la ruta completa de salida
   ruta_salida = os.path.join('..', 'data', 'coocked', 'json', 'scoring', nombre_salida)
   
   # Asegurar que el directorio de salida existe
   os.makedirs(os.path.dirname(ruta_salida), exist_ok=True)
   
   # Cargar productos
   productos = cargar_productos(ruta_archivo)
   
   if productos:
       # Procesar productos y obtener resultados
       resultados = procesar_productos(productos)
       
       # Generar salida en formato JSON
       output_json = json.dumps(resultados, indent=2, ensure_ascii=False)
       print(output_json)
       
       # Guardar resultados en el archivo
       with open(ruta_salida, 'w', encoding='utf-8') as f:
           json.dump(resultados, f, indent=2, ensure_ascii=False)
       print(f"Archivo guardado exitosamente en: {ruta_salida}")

if __name__ == "__main__":
   main()

[
  {
    "product_id": "wzy4p2k8dvz5",
    "description": "Deportivas de mujer Clarks. El Corte Inglés. Piel negra extra suave número 39 1/2 Forradas de piel y con piso de goma. SOLO ENVÍOS ",
    "price": "$30.0",
    "score": 9
  },
  {
    "product_id": "0j247yxnvmzy",
    "description": "Completamente nuevo. 100 % piel. Grande",
    "price": "$150.0",
    "score": 6
  },
  {
    "product_id": "wzv47y7mm7zl",
    "description": "vestido de zara ",
    "price": "$12.0",
    "score": 6
  },
  {
    "product_id": "pzp2e9egk9j3",
    "description": "Hola a todos, bienvenidos a la Sneaker Crew! 👟🌟\nSe vende modelo Nike Air Force 1 Low Shadow \"White Silver Gym Red\" W en talla 42 EU.\n✅ Talla: 42 EU 📏\n✅ Envío rápido y seguro🏃‍♂️📦\n✅ Productos originales 💯\n✅ Ticket o factura disponible 🧾\nPara más detalles o fotos, no dudes en enviarme un mensaje. Gracias por tu interés! 😊",
    "price": "$120.0",
    "score": 10
  },
  {
    "product_id": "vjrq2153g8zk",
    "description": "Nuevo Tiss

Data_motos

In [11]:
import json
import os

def evaluar_descripcion(descripcion):
   score = 0
   # Evaluación por longitud de la descripción
   if len(descripcion) > 150:
       score += 5
   elif len(descripcion) > 100:
       score += 4
   elif len(descripcion) > 50:
       score += 3
   else:
       score += 1

   # Evaluación por palabras clave
   palabras_clave = ["nuevo", "original", "excelente"]
   if any(keyword in descripcion.lower() for keyword in palabras_clave):
       score += 2

   return score

def evaluar_precio(precio):
   if precio < 1000:
       return 5
   elif precio < 5000:
       return 4  
   elif precio < 10000:
       return 3
   elif precio < 20000:
       return 2
   else:
       return 1

def cargar_productos(ruta_archivo):
   try:
       with open(ruta_archivo, 'r', encoding='utf-8') as file:
           return json.load(file)
   except FileNotFoundError:
       print(f"Error: No se encontró el archivo en la ruta {ruta_archivo}")
       return []
   except json.JSONDecodeError:
       print("Error: El archivo no contiene un JSON válido")
       return []

def procesar_productos(productos):
   resultados = []
   for producto in productos:
       descripcion_score = evaluar_descripcion(producto["description"])
       precio_score = evaluar_precio(producto["amount"])
       puntaje_total = min(descripcion_score + precio_score, 10)
       
       resultados.append({
           "product_id": producto["id"],
           "description": producto["description"],
           "price": f"${producto['amount']}",
           "score": puntaje_total
       })
   return resultados

def main():
   # Ruta del archivo JSON de entrada
   ruta_archivo = '..\data\coocked\json\data_motos.json'
   
   # Obtener el nombre base del archivo sin extensión y sin ruta
   nombre_base = os.path.basename(ruta_archivo)
   nombre_sin_extension = os.path.splitext(nombre_base)[0]
   
   # Crear el nombre del archivo de salida
   nombre_salida = f"{nombre_sin_extension}.json"
   
   # Crear la ruta completa de salida
   ruta_salida = os.path.join('..', 'data', 'coocked', 'json', 'scoring', nombre_salida)
   
   # Asegurar que el directorio de salida existe
   os.makedirs(os.path.dirname(ruta_salida), exist_ok=True)
   
   # Cargar productos
   productos = cargar_productos(ruta_archivo)
   
   if productos:
       # Procesar productos y obtener resultados
       resultados = procesar_productos(productos)
       
       # Generar salida en formato JSON
       output_json = json.dumps(resultados, indent=2, ensure_ascii=False)
       print(output_json)
       
       # Guardar resultados en el archivo
       with open(ruta_salida, 'w', encoding='utf-8') as f:
           json.dump(resultados, f, indent=2, ensure_ascii=False)
       print(f"Archivo guardado exitosamente en: {ruta_salida}")

if __name__ == "__main__":
   main()

[
  {
    "product_id": "mzn21x5782zn",
    "description": "24.100kms\nMantenimientos en casa oficial segun manual, y cambios de aceite cada 5.000 kms. Tengo todas las facturas.\nTiene garantía oficial\n**Se pueden enviar más fotos si es necesario**\n\nExtras:\nEscape SC crt titanio\nEspejos Rizoma\nProtector Basculante entero y Cadena de carbono\nProtectores de maneras\nProtectores de Basculante\nProtector depósito carbono\nColín monoplaza original Aprilia\nCupula doble burbuja Puig\nIntermitentes dinámicos\nPosibilidad de entregae con todo original.\n\nSe ha cuidado mucho la moto, duerme en garaje y nunca ha pisado circuito.\n**NO SE VENDEN PIEZAS POR SEPARADO**",
    "price": "$9900",
    "score": 10
  },
  {
    "product_id": "xzo248odk569",
    "description": "\n          KAWASAKI Z 900 A2\n    - Año: 2021\n    - Kilómetros: 24691\n    - Garantía: 1 año\n    - Precio: 7199 (no negociable)\n    - Tipo: NAKED\n    - Carnet: A / A2\n\n    // Entregamos la moto totalmente revisada y c

data_niños

In [12]:
import json
import os

def evaluar_descripcion(descripcion):
   score = 0
   # Evaluación por longitud de la descripción
   if len(descripcion) > 150:
       score += 5
   elif len(descripcion) > 100:
       score += 4
   elif len(descripcion) > 50:
       score += 3
   else:
       score += 1

   # Evaluación por palabras clave
   palabras_clave = ["nuevo", "perfecto estado", "sin estrenar"]
   if any(keyword in descripcion.lower() for keyword in palabras_clave):
       score += 2

   return score

def evaluar_precio(precio):
   if precio < 50:
       return 5
   elif precio < 150:
       return 4
   elif precio < 500:
       return 3
   elif precio < 1000:
       return 2
   else:
       return 1

def cargar_productos(ruta_archivo):
   try:
       with open(ruta_archivo, 'r', encoding='utf-8') as file:
           return json.load(file)
   except FileNotFoundError:
       print(f"Error: No se encontró el archivo en la ruta {ruta_archivo}")
       return []
   except json.JSONDecodeError:
       print("Error: El archivo no contiene un JSON válido")
       return []

def procesar_productos(productos):
   resultados = []
   for producto in productos:
       descripcion_score = evaluar_descripcion(producto["description"])
       precio_score = evaluar_precio(producto["amount"])
       puntaje_total = min(descripcion_score + precio_score, 10)
       
       resultados.append({
           "product_id": producto["id"],
           "description": producto["description"],
           "price": f"${producto['amount']}",
           "score": puntaje_total
       })
   return resultados

def main():
   # Ruta del archivo JSON de entrada
   ruta_archivo = '..\data\coocked\json\data_niños_bebes.json'
   
   # Obtener el nombre base del archivo sin extensión y sin ruta
   nombre_base = os.path.basename(ruta_archivo)
   nombre_sin_extension = os.path.splitext(nombre_base)[0]
   
   # Crear el nombre del archivo de salida
   nombre_salida = f"{nombre_sin_extension}.json"
   
   # Crear la ruta completa de salida
   ruta_salida = os.path.join('..', 'data', 'coocked', 'json', 'scoring', nombre_salida)
   
   # Asegurar que el directorio de salida existe
   os.makedirs(os.path.dirname(ruta_salida), exist_ok=True)
   
   # Cargar productos
   productos = cargar_productos(ruta_archivo)
   
   if productos:
       # Procesar productos y obtener resultados
       resultados = procesar_productos(productos)
       
       # Generar salida en formato JSON
       output_json = json.dumps(resultados, indent=2, ensure_ascii=False)
       print(output_json)
       
       # Guardar resultados en el archivo
       with open(ruta_salida, 'w', encoding='utf-8') as f:
           json.dump(resultados, f, indent=2, ensure_ascii=False)
       print(f"Archivo guardado exitosamente en: {ruta_salida}")

if __name__ == "__main__":
   main()

[
  {
    "product_id": "p614077gy765",
    "description": "Silla de paseo Yoyo de Babyzen en perfecto estado en color Aqua xon capazo a juego. El capazo apenas tiene uso y la silla la vendemos por tener otra. \n\nIncluso el manillar, que a menudo se desgasta con el uso, está impecable.\n",
    "price": "$450.0",
    "score": 10
  },
  {
    "product_id": "mzn2kvv37kzn",
    "description": "En perfecto estado",
    "price": "$5.0",
    "score": 8
  },
  {
    "product_id": "w674oggg7w6x",
    "description": "Sin estrenar",
    "price": "$15.0",
    "score": 8
  },
  {
    "product_id": "8j34wy00l869",
    "description": "Muñeca BRIDGET de Mattel de Descendants 4.\nNUEVA, sin abrir.",
    "price": "$37.0",
    "score": 8
  },
  {
    "product_id": "9jdk939ymn6k",
    "description": "Juguete nunca utilizado “Mi Gran Tienda”, incluye datafono, caja, billetes, monedas y otros accesorios.\n\n+3 años\n\nprecio en el Corte Inglés 39,95 EUR\n\nExtraviamos ticket regalo y no lo pudimos cambiar.

data_otros

In [13]:
import json
import os

def evaluar_descripcion(descripcion):
   score = 0
   # Evaluación por longitud de la descripción
   if len(descripcion) > 150:
       score += 5
   elif len(descripcion) > 100:
       score += 4
   elif len(descripcion) > 50:
       score += 3
   else:
       score += 1

   # Evaluación por palabras clave
   palabras_clave = ["nuevo", "perfecto estado", "sin estrenar"]
   if any(keyword in descripcion.lower() for keyword in palabras_clave):
       score += 2

   return score

def evaluar_precio(precio):
   if precio < 50:
       return 5
   elif precio < 150:
       return 4
   elif precio < 500:
       return 3
   elif precio < 1000:
       return 2
   else:
       return 1

def cargar_productos(ruta_archivo):
   try:
       with open(ruta_archivo, 'r', encoding='utf-8') as file:
           return json.load(file)
   except FileNotFoundError:
       print(f"Error: No se encontró el archivo en la ruta {ruta_archivo}")
       return []
   except json.JSONDecodeError:
       print("Error: El archivo no contiene un JSON válido")
       return []

def procesar_productos(productos):
   resultados = []
   for producto in productos:
       descripcion_score = evaluar_descripcion(producto["description"])
       precio_score = evaluar_precio(producto["amount"])
       puntaje_total = min(descripcion_score + precio_score, 10)
       
       resultados.append({
           "product_id": producto["id"],
           "description": producto["description"],
           "price": f"{producto['amount']} {producto['currency']}",
           "score": puntaje_total
       })
   return resultados

def main():
   # Ruta del archivo JSON de entrada
   ruta_archivo = '..\data\coocked\json\data_otros.json'
   
   # Obtener el nombre base del archivo sin extensión y sin ruta
   nombre_base = os.path.basename(ruta_archivo)
   nombre_sin_extension = os.path.splitext(nombre_base)[0]
   
   # Crear el nombre del archivo de salida
   nombre_salida = f"{nombre_sin_extension}.json"
   
   # Crear la ruta completa de salida
   ruta_salida = os.path.join('..', 'data', 'coocked', 'json', 'scoring', nombre_salida)
   
   # Asegurar que el directorio de salida existe
   os.makedirs(os.path.dirname(ruta_salida), exist_ok=True)
   
   # Cargar productos
   productos = cargar_productos(ruta_archivo)
   
   if productos:
       # Procesar productos y obtener resultados
       resultados = procesar_productos(productos)
       
       # Generar salida en formato JSON
       output_json = json.dumps(resultados, indent=2, ensure_ascii=False)
       print(output_json)
       
       # Guardar resultados en el archivo
       with open(ruta_salida, 'w', encoding='utf-8') as f:
           json.dump(resultados, f, indent=2, ensure_ascii=False)
       print(f"Archivo guardado exitosamente en: {ruta_salida}")

if __name__ == "__main__":
   main()

[
  {
    "product_id": "nz04o7v4x7jo",
    "description": "Figura de Sukuna, de Fitguars Zero , anime jujutsu kaisen, medidas 13 cm aprox",
    "price": "89.95 EUR",
    "score": 7
  },
  {
    "product_id": "pzp2e11e5kj3",
    "description": "perfecto 👌722655568",
    "price": "65.0 EUR",
    "score": 5
  },
  {
    "product_id": "3zl82g5g4p6x",
    "description": "foco led con lente azul se utiliza para proyectar la luz sobre las boquillas de barras de tratamiento agricola para ir verificando que todas las boquillas arrojan producto sobre el terreno, 18w de potencia funciona a 12v y 24v muy practico, enviamos a toda españa contarembolso atendemos whattsap telefono email los 365 dias al año. ademas contamos con todo tipo de focos led y barras 18w 27w 48w 72w 96w 120w 180w 240w 300w. se envia segovia la coruña leon granada asturias burgos huesca palencia valladolidetc",
    "price": "30.0 EUR",
    "score": 10
  },
  {
    "product_id": "nz04pmd9yrjo",
    "description": "NUEVOS, Foco

data_tecnologia

In [14]:
import json
import os

def evaluar_descripcion(descripcion):
   score = 0
   # Evaluación por longitud de la descripción
   if len(descripcion) > 150:
       score += 5
   elif len(descripcion) > 100:
       score += 4
   elif len(descripcion) > 50:
       score += 3
   else:
       score += 1

   # Evaluación por palabras clave
   palabras_clave = ["nuevo", "perfecto estado", "sin estrenar"]  
   if any(keyword in descripcion.lower() for keyword in palabras_clave):
       score += 2

   return score

def evaluar_precio(precio):
   if precio < 50:
       return 5
   elif precio < 150:
       return 4
   elif precio < 500:
       return 3
   elif precio < 1000:
       return 2
   else:
       return 1

def cargar_productos(ruta_archivo):
   try:
       with open(ruta_archivo, 'r', encoding='utf-8') as file:
           return json.load(file)
   except FileNotFoundError:
       print(f"Error: No se encontró el archivo en la ruta {ruta_archivo}")
       return []
   except json.JSONDecodeError:
       print("Error: El archivo no contiene un JSON válido")
       return []

def procesar_productos(productos):
   resultados = []
   for producto in productos:
       descripcion_score = evaluar_descripcion(producto["description"])
       precio_score = evaluar_precio(producto["amount"])
       puntaje_total = min(descripcion_score + precio_score, 10)
       
       resultados.append({
           "product_id": producto["id"],
           "description": producto["description"],
           "price": f"{producto['amount']} {producto['currency']}",
           "score": puntaje_total
       })
   return resultados

def main():
   # Ruta del archivo JSON de entrada
   ruta_archivo = '..\data\coocked\json\data_tecnologia_electronica.json'
   
   # Obtener el nombre base del archivo sin extensión y sin ruta
   nombre_base = os.path.basename(ruta_archivo)
   nombre_sin_extension = os.path.splitext(nombre_base)[0]
   
   # Crear el nombre del archivo de salida
   nombre_salida = f"{nombre_sin_extension}.json"
   
   # Crear la ruta completa de salida
   ruta_salida = os.path.join('..', 'data', 'coocked', 'json', 'scoring', nombre_salida)
   
   # Asegurar que el directorio de salida existe
   os.makedirs(os.path.dirname(ruta_salida), exist_ok=True)
   
   # Cargar productos
   productos = cargar_productos(ruta_archivo)
   
   if productos:
       # Procesar productos y obtener resultados
       resultados = procesar_productos(productos)
       
       # Generar salida en formato JSON
       output_json = json.dumps(resultados, indent=2, ensure_ascii=False)
       print(output_json)
       
       # Guardar resultados en el archivo
       with open(ruta_salida, 'w', encoding='utf-8') as f:
           json.dump(resultados, f, indent=2, ensure_ascii=False)
       print(f"Archivo guardado exitosamente en: {ruta_salida}")

if __name__ == "__main__":
   main()

[
  {
    "product_id": "9jdk0x5pw46k",
    "description": "macbook pro M1 max, 32gb ram 1tb ssd de almacenamiento, comprado en Agosto de 2022, esta muy bien cuidado soy ingeniero y necesito renovarlo ya, incluye la caja y sus accesorios originales.La bateria esta de salud normal al 92%. Casi siempre ha estado en mi casa y se ha movido poco, el estado es impecable, por favor no me interesan cambios ni ofertas absurdas y sou preferencia a entrega en mano en Madrid para probarlo y que se compruebe que todo esta correcto os puedo ayudar a configurarlo. Si hay que enviar solo se envia previo pago adelantado. ",
    "price": "1600.0 EUR",
    "score": 6
  },
  {
    "product_id": "wzv47llrm1zl",
    "description": "La television funciona correctamente, con tdt incluido ",
    "price": "50.0 EUR",
    "score": 7
  },
  {
    "product_id": "e65813yr006o",
    "description": "Juego de PS4 en perfectas condiciones. Solamente envío. Se hacen lotes ",
    "price": "29.0 EUR",
    "score": 8
  },


data_servicios

In [15]:
import json
import os

def evaluar_descripcion(descripcion):
   score = 0
   # Evaluación por longitud de la descripción
   if len(descripcion) > 150:
       score += 5
   elif len(descripcion) > 100:
       score += 4
   elif len(descripcion) > 50:
       score += 3
   else:
       score += 1

   # Evaluación por palabras clave
   palabras_clave = ["nuevo", "perfecto estado", "sin estrenar"]
   if any(keyword in descripcion.lower() for keyword in palabras_clave):
       score += 2

   return score

def evaluar_precio(precio):
   if precio < 50:
       return 5
   elif precio < 150:
       return 4
   elif precio < 500:
       return 3
   elif precio < 1000:
       return 2
   else:
       return 1

def cargar_productos(ruta_archivo):
   try:
       with open(ruta_archivo, 'r', encoding='utf-8') as file:
           return json.load(file)
   except FileNotFoundError:
       print(f"Error: No se encontró el archivo en la ruta {ruta_archivo}")
       return []
   except json.JSONDecodeError:
       print("Error: El archivo no contiene un JSON válido")
       return []

def procesar_productos(productos):
   resultados = []
   for producto in productos:
       descripcion_score = evaluar_descripcion(producto["description"])
       precio_score = evaluar_precio(producto["amount"])
       puntaje_total = min(descripcion_score + precio_score, 10)
       
       resultados.append({
           "product_id": producto["id"],
           "description": producto["description"],
           "price": f"{producto['amount']} {producto['currency']}",
           "score": puntaje_total
       })
   return resultados

def main():
   # Ruta del archivo JSON de entrada
   ruta_archivo = '..\data\coocked\json\data_servicios.json'
   
   # Obtener el nombre base del archivo sin extensión y sin ruta
   nombre_base = os.path.basename(ruta_archivo)
   nombre_sin_extension = os.path.splitext(nombre_base)[0]
   
   # Crear el nombre del archivo de salida
   nombre_salida = f"{nombre_sin_extension}.json"
   
   # Crear la ruta completa de salida
   ruta_salida = os.path.join('..', 'data', 'coocked', 'json', 'scoring', nombre_salida)
   
   # Asegurar que el directorio de salida existe
   os.makedirs(os.path.dirname(ruta_salida), exist_ok=True)
   
   # Cargar productos
   productos = cargar_productos(ruta_archivo)
   
   if productos:
       # Procesar productos y obtener resultados
       resultados = procesar_productos(productos)
       
       # Generar salida en formato JSON
       output_json = json.dumps(resultados, indent=2, ensure_ascii=False)
       print(output_json)
       
       # Guardar resultados en el archivo
       with open(ruta_salida, 'w', encoding='utf-8') as f:
           json.dump(resultados, f, indent=2, ensure_ascii=False)
       print(f"Archivo guardado exitosamente en: {ruta_salida}")

if __name__ == "__main__":
   main()

[
  {
    "product_id": "36e14q989yjd",
    "description": " ☎️ [ 692-845-484 ] Hacemos mudanzas  puerta a puerta  Montaje de muebles. Cuidamos bien sus muebles. Traslados paqueteria, habitaciones, casas, pisos, apartamentos. Mudanzas baratas - económicas con profesionales. Vaciados de pisos locales, trasteros, garajes pregunta por la Oferta por WhatsApp o telefono 692 84 54 84",
    "price": "99.0 EUR",
    "score": 9
  },
  {
    "product_id": "x6qmg9kpvojy",
    "description": "Soy profesor bilingüe con larga experiencia dando clases de inglés de negocios en distintos sectores (aviación, hoteles, banca privada, escuelas de negocios, retail, marcas de lujo, agencias de publicidad y comunicación). No uso libros ni sigo métodos estandarizados (que es lo que hacen las academias) para que no pierdas el tiempo. Todo el material lo preparo específicamente para cada alumno, con énfasis siempre en conversación. Para tripulaciones de vuelo, atención al público, entrevistas de trabajo, etc. En